In [1]:
from mysklearn import mypytable 
import numpy as np
np.random.seed(0)

this is the file that im going to use to chop up the data  
the datafile is over 6 million lines

i want it to be less than 100k lines  
while also being equally distributed  
between the classes


In [2]:
infile = open("Fraud.csv", "r")
contents = infile.readlines()
infile.close()

In [3]:
print(len(contents))
print(type(contents))
header = contents[0].split(",")
header[-1] = header[-1].strip() # remove the newline (if it is there)
print(header)

6362621
<class 'list'>
['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']


the outputs are:
* step: the hour since the start of the dataset (starts at 1, goes to 744)
* type: {'PAYMENT', 'DEBIT', 'CASH_OUT', 'CASH_IN', 'TRANSFER'}
* amount: the ammount
* nameOrig: name
* oldbalanceOrg: bal
* newbalanceOrig: bal
* nameDest: name
* oldbalanceDest: bal (will be 0.0 if it is a payment, 
* newbalanceDest: bal (will be 0.0 if it is a payment, 
* isFraud: 0, 1
* isFlaggedFraud: 0 1


NOTE:
* i will do the data cleaning (removing the unneeded cols) later

In [4]:
# used while making this notebook

# uniques = {}
# for col in header:
#     index = header.index(col)
#     uniques[col] = set()
#     for line in contents[1:]:
#         line = line.split(",")
#         line[-1] = line[-1].strip() # removing the newline character (if it is there)
#         value = line[index].strip()
#         if line[index] not in uniques[col]:
#             uniques[col].add(value)
# print(uniques["isFlaggedFraud\n"])

In [5]:
data = []
for line in contents[1:]:
    line = line.split(",")
    line[-1] = line[-1].strip() # removing the newline character (if it is there)
    data.append(line)

In [6]:
def get_occur(data):
    fraud_occur = 0
    legit_occur = 0
    for line in data:
        if line[-2] == "1":
            fraud_occur += 1
        else:
            legit_occur += 1
    return fraud_occur, legit_occur

In [7]:
fraud_occur, legit_occur = get_occur(data)
print("There are",legit_occur,"legal transactions")
print("There are",fraud_occur,"fraudulent transactions")
print(legit_occur / fraud_occur, "ratio")

There are 6354407 legal transactions
There are 8213 fraudulent transactions
773.7010836478753 ratio


We want to shrink the dataset size, while keeping the same ratio  
also to make life easier, i will be rounding the ratio to be 774 : 1  

For every 774 legal transactions, there is a fradulant transaction

In [8]:
legal = []
fraudulent = []
for line in data:
    if line[-2] == "1":
        fraudulent.append(line)
    else:
        legal.append(line)

# just double checking
assert len(legal) + len(fraudulent) == len(data)
assert len(legal) == legit_occur
assert len(fraudulent) == fraud_occur

In [9]:
# becayse for every 774 legal transactions, there is a fraudulent transaction
# we can have the dataset be however many fradulent lines we want, with that number times 774 legal ones

new_data = []
# num_fradulent = len(fraudulent) // 600
num_fradulent = 500
num_legal = num_fradulent * 774

for i in range(num_fradulent):
    # a faster, but not random way to do this
    # rand_index = i
    # new_data.append(fraudulent[rand_index])

    # a slower, but random way to do thi
    rand_index = np.random.randint(0, len(fraudulent))
    new_data.append(fraudulent.pop(rand_index))
for i in range(num_legal):
    # a faster, but not random way to do this
    # rand_index = i
    # new_data.append(legal[rand_index])
    
    # a slower, but random way to do this
    rand_index = np.random.randint(0, len(legal))
    new_data.append(legal.pop(rand_index))

fraud_occur, legit_occur = get_occur(new_data)
print("There are",legit_occur,"legal transactions")
print("There are",fraud_occur,"fraudulent transactions")
print(legit_occur / fraud_occur, "ratio")

There are 387000 legal transactions
There are 500 fraudulent transactions
774.0 ratio


In [10]:
table = mypytable.MyPyTable(column_names=header, data=new_data)
table.save_to_file("Fraud_chop.csv")